# Notes
- simple article: https://www.geeksforgeeks.org/introduction-to-beam-search-algorithm/
- many beam search descriptions are given in terms of NLP context
- I think A* might not be feasible for large graphs, but I'm not sure...
- It'd be nice to code up the Clifford problem in a nice object-oriented with that made the graph structure abstracted, so I could use any old graph traversal algorithm

In [1]:
import tqdm
from typing import Dict
import numpy as np
import torch
import torch.nn as nn
import rubiks.clifford as cl
import rubiks.lgf as lgf
from qiskit.quantum_info import Clifford

In [2]:
num_qubits = 3

SEED = 123
use_qiskit = False
device = torch.device('cpu')
drop_phase_bits = True
scaling = 'log-linear'
data_dir = f"data/data_n_{num_qubits}_drop_phase_bits_scaling_{scaling}/"
high = cl.max_random_sequence_length(num_qubits, scaling)

lgf_model = lgf.LGFModel(
    num_qubits=num_qubits,
    device=device,
    rng=np.random.default_rng(SEED),
    hidden_layers=[32, 16, 4, 2],
    drop_phase_bits=drop_phase_bits,
    use_qiskit=use_qiskit,
).to(device)

lgf_model.load_state_dict(torch.load(data_dir + "checkpoint"))

<All keys matched successfully>

- Put this in the same format as other hillclimbing function
- Compare for different values of beam_width
    - should agree with previous results for beam_width=1
    - should be better for larger beam_widths
- Can it be made faster?

In [3]:
sequence_length = 15
initial_sequence = cl.random_sequence(np.random.default_rng(), seq_length=sequence_length, num_qubits=num_qubits)
initial_state = cl.sequence_to_tableau(initial_sequence, num_qubits)
results_dict = lgf.beam_search(initial_state=initial_state, lgf_model=lgf_model, beam_width=3, max_count=100)

count = results_dict['count']
success = results_dict['success']

if success is None:
    print(f"No solution found after {count} steps")
else:
    print(f"Solution found after {count} steps")

Solution found after 11 steps


In [4]:
len(results_dict['linked_list'].get_move_list()[:-1])

12

In [5]:
results_dict['linked_list'].get_move_list()[:-1]

[(0, 'x'),
 (1, 'y'),
 (2, 'x'),
 (0, 'z'),
 (1, 'z'),
 (2, 'y'),
 (1, 'y'),
 (1, 'sdg'),
 (1, 'h'),
 (2, 'h'),
 (0, 1, 'cx'),
 (0, 2, 'swap')]

In [ ]:
initial_state @ cl.sequence_to_tableau(results_dict['linked_list'].get_move_list()[:-1][::-1], num_qubits=num_qubits)